In [1]:
from config import API_KEY

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

from sqlalchemy import create_engine
import pandas as pd

In [5]:
# Create API objects
ts = TimeSeries(key=API_KEY, output_format='pandas')
ti = TechIndicators(key=API_KEY, output_format='pandas')

# Set db variables
user = 'anthony'
pw = 'pw'
host = 'localhost:5432'
db_name = 'indicator_tests'

# Create engine for interacting with db
engine = create_engine(f'postgresql+psycopg2://{user}:{pw}@{host}/{db_name}')

In [6]:
# Set API variables
ticker = 'NTNX'
interval = '5min'
outputsize = 'full'

In [7]:
# Get stock price data
df, _ = ts.get_intraday(ticker, interval=interval, outputsize=outputsize)

In [8]:
df.drop(columns='5. volume', inplace=True)
df.sort_index(ascending=True, inplace=True)

df.rename(columns={"1. open": "open",
                   "2. high": "high",
                   "3. low": "low",
                   "4. close": "close"}, 
          inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1168 entries, 2019-08-20 09:35:00 to 2019-09-10 15:50:00
Data columns (total 4 columns):
open     1168 non-null float64
high     1168 non-null float64
low      1168 non-null float64
close    1168 non-null float64
dtypes: float64(4)
memory usage: 45.6+ KB


In [11]:
# Get sma data
sma8, _ = ti.get_sma(ticker, interval=interval, time_period=8, series_type='close')
sma13, _ = ti.get_sma(ticker, interval=interval, time_period=13, series_type='close')


In [12]:
sma8.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1154 entries, 2019-08-20 10:10 to 2019-09-10 15:50
Data columns (total 1 columns):
SMA    1154 non-null float64
dtypes: float64(1)
memory usage: 18.0+ KB


In [14]:
sma13.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1149 entries, 2019-08-20 10:35 to 2019-09-10 15:50
Data columns (total 1 columns):
SMA    1149 non-null float64
dtypes: float64(1)
memory usage: 18.0+ KB


In [19]:
# Match index range to the ntnx data

sma8 = sma8[sma8.index >= '2019-08-27 10:45']
sma13 = sma13[sma13.index >= '2019-08-27 10:45']

,open,high,low,close
date,,,,
2019-08-20 09:35:00,18.9700,19.1200,18.6500,18.6600
2019-08-20 09:40:00,18.7203,19.0365,18.7062,19.0033
2019-08-20 09:45:00,18.9800,19.0000,18.8300,18.9200
2019-08-20 09:50:00,18.8971,18.9200,18.8400,18.8550
2019-08-20 09:55:00,18.8605,18.9183,18.8282,18.8905
